In [1]:
import pandas as pd

In [2]:
from fugue_notebook import setup


C:\Users\phamp\AppData\Roaming\Python\Python39\site-packages\fugue_sql\__init__.py:8: UserWarning: fsql and FugueSQLWorkflow now should be imported directly from fugue, fugue_sql will be removed in 0.9.0
  warnings.warn(


In [3]:
# cấu hình
setup()

<IPython.core.display.Javascript object>

In [4]:
df = pd.read_csv('data/insurance.csv')
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [5]:
# thống kê số lượng nam/nữ
df['sex'].value_counts()

male      676
female    662
Name: sex, dtype: int64

In [6]:
# Sắp xếp dữ liệu giảm dần theo sex
df.sort_values(by='sex', ascending=False)

,age,sex,bmi,children,smoker,region,charges
446,60,male,29.64,0,no,northeast,12730.9996
1052,49,male,29.83,1,no,northeast,9288.0267
1070,37,male,37.07,1,yes,southeast,39871.7043
550,63,male,30.80,0,no,southwest,13390.5590
1068,63,male,21.66,1,no,northwest,14349.8544
...,...,...,...,...,...,...,...
602,56,female,25.30,0,no,southwest,11070.5350
603,64,female,39.05,3,no,southeast,16085.1275
604,19,female,28.31,0,yes,northwest,17468.9839
605,51,female,34.10,0,no,southeast,9283.5620


In [7]:
# Sắp xếp dữ liệu giảm dần theo sex sử dụng fugueSQL

In [8]:
%%fsql
SELECT * FROM df ORDER BY sex DESC
PRINT 

,age:long,sex:str,bmi:double,children:long,smoker:str,region:str,charges:double
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
8,37,male,29.830,2,no,northeast,6406.41070
10,25,male,26.220,0,no,northeast,2721.32080
12,23,male,34.400,0,no,southwest,1826.84300
14,27,male,42.130,0,yes,southeast,39611.75770
15,19,male,24.600,1,no,southwest,1837.23700
17,23,male,23.845,0,no,northeast,2395.17155


In [9]:
# Sắp xếp dữ liệu giảm dần theo sex và tăng dần theo charges (dùng DataFrame)
df.sort_values(by=['sex', 'charges'], ascending=[False, True])

,age,sex,bmi,children,smoker,region,charges
940,18,male,23.210,0,no,southeast,1121.87390
808,18,male,30.140,0,no,southeast,1131.50660
1244,18,male,33.330,0,no,southeast,1135.94070
663,18,male,33.660,0,no,southeast,1136.39940
22,18,male,34.100,0,no,southeast,1137.01100
...,...,...,...,...,...,...,...
175,63,female,37.700,0,yes,southwest,48824.45000
488,44,female,38.060,0,yes,southeast,48885.13561
819,33,female,35.530,0,yes,northwest,55135.40209
577,31,female,38.095,1,yes,northeast,58571.07448


In [10]:
# Sắp xếp dữ liệu giảm dần theo sex và tăng dần theo charges (dùng fugueSQL)

In [11]:
%%fsql
SELECT *
FROM df
ORDER BY sex DESC, charges
PRINT

,age:long,sex:str,bmi:double,children:long,smoker:str,region:str,charges:double
940,18,male,23.21,0,no,southeast,1121.8739
808,18,male,30.14,0,no,southeast,1131.5066
1244,18,male,33.33,0,no,southeast,1135.9407
663,18,male,33.66,0,no,southeast,1136.3994
22,18,male,34.10,0,no,southeast,1137.0110
194,18,male,34.43,0,no,southeast,1137.4697
866,18,male,37.29,0,no,southeast,1141.4451
781,18,male,41.14,0,no,southeast,1146.7966
442,18,male,43.01,0,no,southeast,1149.3959
1317,18,male,53.13,0,no,southeast,1163.4627


In [12]:
# cho biết chỉ số bmi trung bình theo sex (sử dụng DataFrame)
df.groupby(by=['sex'], as_index=False)['bmi'].mean()

,sex,bmi
0,female,30.377749
1,male,30.943129


In [13]:
# cho biết chỉ số bmi trung bình theo sex (sử dụng fugueSQL)

In [14]:
%%fsql
SELECT sex, AVG(bmi) AS avg_bmi
FROM df
GROUP BY sex
PRINT

,sex:str,avg_bmi:double
0,female,30.377749
1,male,30.943129


In [15]:
# Tạo bảng chứa danh sách các nhân viên gồm 3 thông tin: Mã số, Tên, Tuổi
# 'A0001', 'Nguyễn Trần Lê', 25
# 'A0002', 'Nguyễn Lê Trần', 20
# 'A0003', 'Lê Trần Nguyễn', 28

In [16]:
%%fsql
CREATE [['A0001', 'Nguyễn Trần Lê', 25], ['A0002', 'Nguyễn Lê Trần', 20], ['A0003', 'Lê Trần Nguyễn', 28]]
SCHEMA maso:str, hoten:str, tuoi:int
PRINT

,maso:str,hoten:str,tuoi:int
0,A0001,Nguyễn Trần Lê,25
1,A0002,Nguyễn Lê Trần,20
2,A0003,Lê Trần Nguyễn,28


In [17]:
# tạo bảng từ biến kiểu list
myData = [['A0001', 'Nguyễn Trần Lê', 25], ['A0002', 'Nguyễn Lê Trần', 20], ['A0003', 'Lê Trần Nguyễn', 28]]

In [18]:
%%fsql
CREATE {{myData}}
SCHEMA maso:str, hoten:str, tuoi:int
PRINT

,maso:str,hoten:str,tuoi:int
0,A0001,Nguyễn Trần Lê,25
1,A0002,Nguyễn Lê Trần,20
2,A0003,Lê Trần Nguyễn,28


In [19]:
import seaborn as sns

In [20]:
dfIris = sns.load_dataset('iris')
dfIris.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [21]:
myData = dfIris.values.tolist()

In [22]:
%%fsql
CREATE {{myData}}
SCHEMA sepallength:double, sepalwidth:double, petallength:double, petalwidth:double, species:str
PRINT

,sepallength:double,sepalwidth:double,petallength:double,petalwidth:double,species:str
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
5,5.4,3.9,1.7,0.4,setosa
6,4.6,3.4,1.4,0.3,setosa
7,5.0,3.4,1.5,0.2,setosa
8,4.4,2.9,1.4,0.2,setosa
9,4.9,3.1,1.5,0.1,setosa


In [23]:
# tạo bảng bằng cách load trực tiếp từ tập tin

In [24]:
%%fsql
tmp = LOAD "data/insurance.csv" (header=true, infer_schema=TRUE)
SELECT * FROM tmp
PRINT

,age:long,sex:str,bmi:double,children:long,smoker:str,region:str,charges:double
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
5,31,female,25.740,0,no,southeast,3756.62160
6,46,female,33.440,1,no,southeast,8240.58960
7,37,female,27.740,3,no,northwest,7281.50560
8,37,male,29.830,2,no,northeast,6406.41070
9,60,female,25.840,0,no,northwest,28923.13692


In [25]:
# cho biết danh sách khách hàng nữ tham gia bảo hiểm và có hút thuốc

In [26]:
%%fsql
tmp = LOAD "data/insurance.csv" (header=true, infer_schema=TRUE)
SELECT * 
FROM tmp
WHERE sex='female' AND smoker='yes'
PRINT

,age:long,sex:str,bmi:double,children:long,smoker:str,region:str,charges:double
0,19,female,27.90,0,yes,southwest,16884.9240
11,62,female,26.29,0,yes,southeast,27808.7251
23,34,female,31.92,1,yes,northeast,37701.8768
58,53,female,22.88,1,yes,southeast,23244.7902
64,20,female,22.42,0,yes,northwest,14711.7438
70,27,female,24.75,0,yes,southeast,16577.7795
84,37,female,34.80,2,yes,southwest,39836.5190
86,57,female,31.16,0,yes,northwest,43578.9394
94,64,female,31.30,2,yes,southwest,47291.0550
103,61,female,29.92,3,yes,southeast,30942.1918


In [27]:
# cho biết bmi trung bình và tổng charges theo giới tính và có hút thuốc hay không

In [28]:
%%fsql
tmp = LOAD "data/insurance.csv" (header=true, infer_schema=TRUE)
SELECT sex, smoker, AVG(bmi) AS avg_bmi, SUM(charges) AS sum_charges
FROM tmp
GROUP BY sex, smoker
PRINT

,sex:str,smoker:str,avg_bmi:double,sum_charges:double
0,female,no,30.539525,4.792977e+06
1,female,yes,29.608261,3.528085e+06
2,male,no,30.770580,4.181085e+06
3,male,yes,31.504182,5.253679e+06


In [29]:
dfTips = sns.load_dataset('tips')
dfTips.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [30]:
dfTips.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB


In [31]:
dfTips['sex'] = dfTips['sex'].astype(str)
dfTips['smoker'] = dfTips['smoker'].astype(str)
dfTips['day'] = dfTips['day'].astype(str)
dfTips['time'] = dfTips['time'].astype(str)

In [32]:
dfTips.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   total_bill  244 non-null    float64
 1   tip         244 non-null    float64
 2   sex         244 non-null    object 
 3   smoker      244 non-null    object 
 4   day         244 non-null    object 
 5   time        244 non-null    object 
 6   size        244 non-null    int64  
dtypes: float64(2), int64(1), object(4)
memory usage: 13.5+ KB


In [33]:
# Liệt kê danh sách thực khách nữ (dùng fugueSQL)

In [34]:
%%fsql
SELECT *
FROM dfTips
WHERE sex='Female'
PRINT

,total_bill:double,tip:double,sex:str,smoker:str,day:str,time:str,size:long
0,16.99,1.01,Female,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
11,35.26,5.00,Female,No,Sun,Dinner,4
14,14.83,3.02,Female,No,Sun,Dinner,2
16,10.33,1.67,Female,No,Sun,Dinner,3
18,16.97,3.50,Female,No,Sun,Dinner,3
21,20.29,2.75,Female,No,Sat,Dinner,2
22,15.77,2.23,Female,No,Sat,Dinner,2
29,19.65,3.00,Female,No,Sat,Dinner,2
32,15.06,3.00,Female,No,Sat,Dinner,2


In [35]:
# Liệt kê danh sách 5 thực khách nữ có hút thuốc với số tiền thanh toán nhiều nhất (dùng fugueSQL)

In [36]:
%%fsql
SELECT *
FROM dfTips
WHERE sex='Female' AND smoker='Yes'
ORDER BY total_bill DESC
LIMIT 5
PRINT

,total_bill:double,tip:double,sex:str,smoker:str,day:str,time:str,size:long
102,44.30,2.50,Female,Yes,Sat,Dinner,3
197,43.11,5.00,Female,Yes,Thur,Lunch,4
219,30.14,3.09,Female,Yes,Sat,Dinner,4
214,28.17,6.50,Female,Yes,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2


In [37]:
# Thống kê tổng số tiền thanh toán theo ngày, chỉ liệt kê các ngày có tổng tiền thanh toán > 1000

In [38]:
%%fsql
SELECT day, SUM(total_bill) AS sum_totalbill
FROM dfTips
GROUP BY day
HAVING SUM(total_bill) > 1000
PRINT

,day:str,sum_totalbill:double
1,Sat,1778.40
2,Sun,1627.16
3,Thur,1096.33


In [39]:
# Liệt kê thông tin các bữa ăn kèm theo số tiền thanh toán trung bình và tiền tip trung bình, gồm:
# total_bill, tip, day, avg_totalbill, avg_tip

In [40]:
%%fsql
SELECT total_bill, tip, day, AVG(total_bill) OVER() AS avg_totalbill, AVG(tip) OVER() AS avg_tip
FROM dfTips
PRINT

,total_bill:double,tip:double,day:str,avg_totalbill:double,avg_tip:double
0,16.99,1.01,Sun,19.785943,2.998279
1,10.34,1.66,Sun,19.785943,2.998279
2,21.01,3.50,Sun,19.785943,2.998279
3,23.68,3.31,Sun,19.785943,2.998279
4,24.59,3.61,Sun,19.785943,2.998279
5,25.29,4.71,Sun,19.785943,2.998279
6,8.77,2.00,Sun,19.785943,2.998279
7,26.88,3.12,Sun,19.785943,2.998279
8,15.04,1.96,Sun,19.785943,2.998279
9,14.78,3.23,Sun,19.785943,2.998279


In [41]:
# Liệt kê thông tin các bữa ăn kèm tổng số tiền thanh toán theo sex, gồm:
# sex, total_bill, tip, sum_totalbill

In [42]:
%%fsql
SELECT sex, total_bill, tip, SUM(total_bill) OVER(PARTITION BY sex) AS sum_totalbill
FROM dfTips
PRINT

,sex:str,total_bill:double,tip:double,sum_totalbill:double
0,Female,16.99,1.01,1570.95
1,Male,10.34,1.66,3256.82
2,Male,21.01,3.50,3256.82
3,Male,23.68,3.31,3256.82
4,Female,24.59,3.61,1570.95
5,Male,25.29,4.71,3256.82
6,Male,8.77,2.00,3256.82
7,Male,26.88,3.12,3256.82
8,Male,15.04,1.96,3256.82
9,Male,14.78,3.23,3256.82


In [43]:
# Liệt kê thông tin các bữa ăn có tiền tip > tiền tip trung bình

In [44]:
%%fsql
dfTemp = SELECT *, AVG(tip) OVER() AS avg_tip
FROM dfTips
SELECT * 
FROM dfTemp
WHERE tip > avg_tip
PRINT

,total_bill:double,tip:double,sex:str,smoker:str,day:str,time:str,size:long,avg_tip:double
2,21.01,3.50,Male,No,Sun,Dinner,3,2.998279
3,23.68,3.31,Male,No,Sun,Dinner,2,2.998279
4,24.59,3.61,Female,No,Sun,Dinner,4,2.998279
5,25.29,4.71,Male,No,Sun,Dinner,4,2.998279
7,26.88,3.12,Male,No,Sun,Dinner,4,2.998279
9,14.78,3.23,Male,No,Sun,Dinner,2,2.998279
11,35.26,5.00,Female,No,Sun,Dinner,4,2.998279
13,18.43,3.00,Male,No,Sun,Dinner,4,2.998279
14,14.83,3.02,Female,No,Sun,Dinner,2,2.998279
15,21.58,3.92,Male,No,Sun,Dinner,2,2.998279


In [45]:
#Liệt kê các bữa ăn có số tiền thanh toán > giá trị do người dùng chỉ định
thanhtoan = 40

In [46]:
%%fsql
SELECT * 
FROM dfTips
WHERE total_bill > {{thanhtoan}}
PRINT

,total_bill:double,tip:double,sex:str,smoker:str,day:str,time:str,size:long
59,48.27,6.73,Male,No,Sat,Dinner,4
95,40.17,4.73,Male,Yes,Fri,Dinner,4
102,44.30,2.50,Female,Yes,Sat,Dinner,3
142,41.19,5.00,Male,No,Thur,Lunch,5
156,48.17,5.00,Male,No,Sun,Dinner,6
170,50.81,10.00,Male,Yes,Sat,Dinner,3
182,45.35,3.50,Male,Yes,Sun,Dinner,3
184,40.55,3.00,Male,Yes,Sun,Dinner,2
197,43.11,5.00,Female,Yes,Thur,Lunch,4
212,48.33,9.00,Male,No,Sat,Dinner,4


In [47]:
# biến cục bộ

In [48]:
%%fsql
dfTemp = SELECT *, AVG(tip) OVER() AS avg_tip
FROM dfTips
SELECT * 
FROM dfTemp
WHERE tip > avg_tip
PRINT

,total_bill:double,tip:double,sex:str,smoker:str,day:str,time:str,size:long,avg_tip:double
2,21.01,3.50,Male,No,Sun,Dinner,3,2.998279
3,23.68,3.31,Male,No,Sun,Dinner,2,2.998279
4,24.59,3.61,Female,No,Sun,Dinner,4,2.998279
5,25.29,4.71,Male,No,Sun,Dinner,4,2.998279
7,26.88,3.12,Male,No,Sun,Dinner,4,2.998279
9,14.78,3.23,Male,No,Sun,Dinner,2,2.998279
11,35.26,5.00,Female,No,Sun,Dinner,4,2.998279
13,18.43,3.00,Male,No,Sun,Dinner,4,2.998279
14,14.83,3.02,Female,No,Sun,Dinner,2,2.998279
15,21.58,3.92,Male,No,Sun,Dinner,2,2.998279


In [49]:
dfTemp

NameError: name 'dfTemp' is not defined

In [50]:
df = pd.read_csv('data/insurance.csv')

In [51]:
%%fsql
SELECT *
FROM df
LIMIT 10
PRINT

,age:long,sex:str,bmi:double,children:long,smoker:str,region:str,charges:double
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
5,31,female,25.740,0,no,southeast,3756.62160
6,46,female,33.440,1,no,southeast,8240.58960
7,37,female,27.740,3,no,northwest,7281.50560
8,37,male,29.830,2,no,northeast,6406.41070
9,60,female,25.840,0,no,northwest,28923.13692


In [52]:
%%fsql
SELECT *
FROM df
ORDER BY charges DESC
LIMIT 10
PRINT

,age:long,sex:str,bmi:double,children:long,smoker:str,region:str,charges:double
543,54,female,47.410,0,yes,southeast,63770.42801
1300,45,male,30.360,0,yes,southeast,62592.87309
1230,52,male,34.485,3,yes,northwest,60021.39897
577,31,female,38.095,1,yes,northeast,58571.07448
819,33,female,35.530,0,yes,northwest,55135.40209
1146,60,male,32.800,0,yes,southwest,52590.82939
34,28,male,36.400,1,yes,southwest,51194.55914
1241,64,male,36.960,2,yes,southeast,49577.66240
1062,59,male,41.140,1,yes,southeast,48970.24760
488,44,female,38.060,0,yes,southeast,48885.13561


In [53]:
%%fsql
SELECT *
FROM df
ORDER BY charges DESC
LIMIT 10
YIELD DATAFRAME AS myDataFrame

In [54]:
myDataFrame

,age:long,sex:str,bmi:double,children:long,smoker:str,region:str,charges:double
543,54,female,47.410,0,yes,southeast,63770.42801
1300,45,male,30.360,0,yes,southeast,62592.87309
1230,52,male,34.485,3,yes,northwest,60021.39897
577,31,female,38.095,1,yes,northeast,58571.07448
819,33,female,35.530,0,yes,northwest,55135.40209
1146,60,male,32.800,0,yes,southwest,52590.82939
34,28,male,36.400,1,yes,southwest,51194.55914
1241,64,male,36.960,2,yes,southeast,49577.66240
1062,59,male,41.140,1,yes,southeast,48970.24760
488,44,female,38.060,0,yes,southeast,48885.13561


In [55]:
type(myDataFrame)

fugue.dataframe.pandas_dataframe.PandasDataFrame

In [56]:
type(df)

pandas.core.frame.DataFrame

In [57]:
myPandasDF = myDataFrame.as_pandas()

In [58]:
type(myPandasDF)

pandas.core.frame.DataFrame

In [59]:
myPandasDF.head()

,age,sex,bmi,children,smoker,region,charges
543,54,female,47.410,0,yes,southeast,63770.42801
1300,45,male,30.360,0,yes,southeast,62592.87309
1230,52,male,34.485,3,yes,northwest,60021.39897
577,31,female,38.095,1,yes,northeast,58571.07448
819,33,female,35.530,0,yes,northwest,55135.40209


In [60]:
# đổi tên cột

In [61]:
%%fsql
SELECT age, sex AS gender, bmi, children, smoker, region, charges
FROM df
ORDER BY charges DESC
LIMIT 10
PRINT

,age:long,gender:str,bmi:double,children:long,smoker:str,region:str,charges:double
543,54,female,47.410,0,yes,southeast,63770.42801
1300,45,male,30.360,0,yes,southeast,62592.87309
1230,52,male,34.485,3,yes,northwest,60021.39897
577,31,female,38.095,1,yes,northeast,58571.07448
819,33,female,35.530,0,yes,northwest,55135.40209
1146,60,male,32.800,0,yes,southwest,52590.82939
34,28,male,36.400,1,yes,southwest,51194.55914
1241,64,male,36.960,2,yes,southeast,49577.66240
1062,59,male,41.140,1,yes,southeast,48970.24760
488,44,female,38.060,0,yes,southeast,48885.13561


In [62]:
%%fsql
dfTemp1 = SELECT *
FROM df
ORDER BY charges DESC
LIMIT 10
dfTemp2 = RENAME COLUMNS sex:gender FROM dfTemp1
PRINT

,age:long,gender:str,bmi:double,children:long,smoker:str,region:str,charges:double
543,54,female,47.410,0,yes,southeast,63770.42801
1300,45,male,30.360,0,yes,southeast,62592.87309
1230,52,male,34.485,3,yes,northwest,60021.39897
577,31,female,38.095,1,yes,northeast,58571.07448
819,33,female,35.530,0,yes,northwest,55135.40209
1146,60,male,32.800,0,yes,southwest,52590.82939
34,28,male,36.400,1,yes,southwest,51194.55914
1241,64,male,36.960,2,yes,southeast,49577.66240
1062,59,male,41.140,1,yes,southeast,48970.24760
488,44,female,38.060,0,yes,southeast,48885.13561


In [ ]:
# đổi kiểu dữ liệu

In [63]:
%%fsql
dfTemp1 = SELECT *
FROM df
ORDER BY charges DESC
LIMIT 10
dfTemp2 = ALTER COLUMNS age:int, bmi:float, children:int FROM dfTemp1
PRINT

,age:int,sex:str,bmi:float,children:int,smoker:str,region:str,charges:double
543,54,female,47.410000,0,yes,southeast,63770.42801
1300,45,male,30.360001,0,yes,southeast,62592.87309
1230,52,male,34.485001,3,yes,northwest,60021.39897
577,31,female,38.095001,1,yes,northeast,58571.07448
819,33,female,35.529999,0,yes,northwest,55135.40209
1146,60,male,32.799999,0,yes,southwest,52590.82939
34,28,male,36.400002,1,yes,southwest,51194.55914
1241,64,male,36.959999,2,yes,southeast,49577.66240
1062,59,male,41.139999,1,yes,southeast,48970.24760
488,44,female,38.060001,0,yes,southeast,48885.13561


In [64]:
%%fsql
dfTemp1 = SELECT *
FROM df
ORDER BY charges DESC
LIMIT 10
dfTemp2 = ALTER COLUMNS age:int, bmi:float, children:int FROM dfTemp1
dfTemp3 = DROP COLUMNS smoker, region IF EXISTS FROM dfTemp2
PRINT

,age:int,sex:str,bmi:float,children:int,charges:double
543,54,female,47.410000,0,63770.42801
1300,45,male,30.360001,0,62592.87309
1230,52,male,34.485001,3,60021.39897
577,31,female,38.095001,1,58571.07448
819,33,female,35.529999,0,55135.40209
1146,60,male,32.799999,0,52590.82939
34,28,male,36.400002,1,51194.55914
1241,64,male,36.959999,2,49577.66240
1062,59,male,41.139999,1,48970.24760
488,44,female,38.060001,0,48885.13561


In [65]:
dfTitanic = sns.load_dataset('titanic')
dfTitanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [66]:
dfTitanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


In [67]:
dfTitanic['class'] = dfTitanic['class'].astype(str)
dfTitanic['deck'] = dfTitanic['deck'].astype(str)

In [68]:
import numpy as np

In [69]:
dfTitanic['deck'] = dfTitanic['deck'].replace('nan', np.NaN)

In [70]:
# Liệt kê các hành khách có deck khác NaN

In [71]:
%%fsql
SELECT *
FROM dfTitanic
WHERE deck != 'nan'
PRINT

,survived:long,pclass:long,sex:str,age:double,sibsp:long,parch:long,fare:double,embarked:str,class:str,who:str,adult_male:bool,deck:str,embark_town:str,alive:str,alone:bool
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
6,0,1,male,54.0,0,0,51.8625,S,First,man,True,E,Southampton,no,True
10,1,3,female,4.0,1,1,16.7000,S,Third,child,False,G,Southampton,yes,False
11,1,1,female,58.0,0,0,26.5500,S,First,woman,False,C,Southampton,yes,True
21,1,2,male,34.0,0,0,13.0000,S,Second,man,True,D,Southampton,yes,True
23,1,1,male,28.0,0,0,35.5000,S,First,man,True,A,Southampton,yes,True
27,0,1,male,19.0,3,2,263.0000,S,First,man,True,C,Southampton,no,False
31,1,1,female,NaN,1,0,146.5208,C,First,woman,False,B,Cherbourg,yes,False
52,1,1,female,49.0,1,0,76.7292,C,First,woman,False,D,Cherbourg,yes,False


In [72]:
%%fsql
SELECT *
FROM dfTitanic
WHERE age IS NOT NULL
PRINT

,survived:long,pclass:long,sex:str,age:double,sibsp:long,parch:long,fare:double,embarked:str,class:str,who:str,adult_male:bool,deck:str,embark_town:str,alive:str,alone:bool
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
6,0,1,male,54.0,0,0,51.8625,S,First,man,True,E,Southampton,no,True
7,0,3,male,2.0,3,1,21.0750,S,Third,child,False,NaN,Southampton,no,False
8,1,3,female,27.0,0,2,11.1333,S,Third,woman,False,NaN,Southampton,yes,False
9,1,2,female,14.0,1,0,30.0708,C,Second,child,False,NaN,Cherbourg,yes,False
10,1,3,female,4.0,1,1,16.7000,S,Third,child,False,G,Southampton,yes,False


In [73]:
%%fsql
dfTemp = SELECT *
FROM dfTitanic
dfTemp2 = DROP ROWS IF ANY NULL ON age FROM dfTemp
PRINT

,survived:long,pclass:long,sex:str,age:double,sibsp:long,parch:long,fare:double,embarked:str,class:str,who:str,adult_male:bool,deck:str,embark_town:str,alive:str,alone:bool
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
5,0,1,male,54.0,0,0,51.8625,S,First,man,True,E,Southampton,no,True
6,0,3,male,2.0,3,1,21.0750,S,Third,child,False,NaN,Southampton,no,False
7,1,3,female,27.0,0,2,11.1333,S,Third,woman,False,NaN,Southampton,yes,False
8,1,2,female,14.0,1,0,30.0708,C,Second,child,False,NaN,Cherbourg,yes,False
9,1,3,female,4.0,1,1,16.7000,S,Third,child,False,G,Southampton,yes,False
